<a href="https://colab.research.google.com/github/znblrean/Automated-Re-Order-Concierge/blob/main/DoctorAppointmentAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi pymongo python-jose[cryptography] passlib[bcrypt] python-dotenv uvicorn pyngrok

In [ ]:
%%writefile project_structure.sh
mkdir -p doctor_appointment_api/{app,config}
touch doctor_appointment_api/app/{main.py,auth.py,appointments.py,models.py,database.py,utils.py}
touch doctor_appointment_api/config/__init__.py doctor_appointment_api/config/settings.py
touch doctor_appointment_api/requirements.txt

Writing project_structure.sh


In [ ]:
!bash project_structure.sh

config/settings.py

In [ ]:
%%writefile doctor_appointment_api/config/settings.py
from pydantic import BaseSettings

class Settings(BaseSettings):
    MONGO_URI: str = "mongodb://localhost:27017"
    DB_NAME: str = "doctor_appointment"
    SECRET_KEY: str = "your-secret-key-here"
    ALGORITHM: str = "HS256"
    ACCESS_TOKEN_EXPIRE_MINUTES: int = 30

    class Config:
        env_file = ".env"

settings = Settings()

Overwriting doctor_appointment_api/config/settings.py


database.py

In [ ]:
%%writefile doctor_appointment_api/app/database.py
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from config.settings import settings

client = None
db = None

def connect_to_mongo():
    global client, db
    try:
        client = MongoClient(settings.MONGO_URI)
        db = client[settings.DB_NAME]
        print("Connected to MongoDB successfully!")
    except ConnectionFailure as e:
        print(f"Could not connect to MongoDB: {e}")

def get_db():
    return db

def close_mongo_connection():
    if client:
        client.close()

Overwriting doctor_appointment_api/app/database.py


models.py

In [ ]:
%%writefile doctor_appointment_api/app/models.py
from datetime import datetime
from pydantic import BaseModel, EmailStr, Field, validator
from typing import Optional
from enum import Enum

class UserCreate(BaseModel):
    email: EmailStr
    password: str = Field(..., min_length=8)

class UserInDB(BaseModel):
    email: str
    hashed_password: str
    created_at: datetime

class UserLogin(BaseModel):
    email: EmailStr
    password: str

class Token(BaseModel):
    access_token: str
    token_type: str

class TokenData(BaseModel):
    email: Optional[str] = None

class AppointmentStatus(str, Enum):
    booked = "booked"
    cancelled = "cancelled"
    completed = "completed"

class AppointmentCreate(BaseModel):
    doctor_id: str
    date: str
    time_slot: str

class AppointmentUpdate(BaseModel):
    date: Optional[str]
    time_slot: Optional[str]

class AppointmentInDB(BaseModel):
    id: str
    user_id: str
    doctor_id: str
    date: str
    time_slot: str
    status: AppointmentStatus
    created_at: datetime
    updated_at: datetime

Overwriting doctor_appointment_api/app/models.py


auth.py

In [ ]:
%%writefile doctor_appointment_api/app/auth.py
from fastapi import APIRouter, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from jose import JWTError, jwt
from passlib.context import CryptContext
from datetime import datetime, timedelta
from typing import Optional
from models import UserCreate, UserInDB, Token, UserLogin
from database import get_db
import pymongo
from config.settings import settings

router = APIRouter(prefix="/api/v1/auth", tags=["auth"])

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="api/v1/auth/signin")

def verify_password(plain_password, hashed_password):
    return pwd_context.verify(plain_password, hashed_password)

def get_password_hash(password):
    return pwd_context.hash(password)

def get_user(email: str):
    db = get_db()
    user = db.users.find_one({"email": email})
    if user:
        return UserInDB(**user)
    return None

def authenticate_user(email: str, password: str):
    user = get_user(email)
    if not user:
        return False
    if not verify_password(password, user.hashed_password):
        return False
    return user

def create_access_token(data: dict, expires_delta: Optional[timedelta] = None):
    to_encode = data.copy()
    if expires_delta:
        expire = datetime.utcnow() + expires_delta
    else:
        expire = datetime.utcnow() + timedelta(minutes=15)
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, settings.SECRET_KEY, algorithm=settings.ALGORITHM)
    return encoded_jwt

@router.post("/signup", response_model=dict)
async def signup(user: UserCreate):
    db = get_db()

    if db.users.find_one({"email": user.email}):
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Email already registered"
        )

    hashed_password = get_password_hash(user.password)
    user_data = {
        "email": user.email,
        "hashed_password": hashed_password,
        "created_at": datetime.utcnow()
    }

    result = db.users.insert_one(user_data)

    return {
        "success": True,
        "message": "User registered successfully.",
        "user_id": str(result.inserted_id)
    }

@router.post("/signin", response_model=Token)
async def signin(form_data: UserLogin):
    user = authenticate_user(form_data.email, form_data.password)
    if not user:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Incorrect email or password",
            headers={"WWW-Authenticate": "Bearer"},
        )

    access_token_expires = timedelta(minutes=settings.ACCESS_TOKEN_EXPIRE_MINUTES)
    access_token = create_access_token(
        data={"sub": user.email}, expires_delta=access_token_expires
    )

    return {"access_token": access_token, "token_type": "bearer"}

Overwriting doctor_appointment_api/app/auth.py


appointments.py

In [ ]:
%%writefile doctor_appointment_api/app/appointments.py
from fastapi import APIRouter, Depends, HTTPException, status
from datetime import datetime
from typing import List
from models import AppointmentCreate, AppointmentUpdate, AppointmentInDB
from auth import oauth2_scheme, get_user_from_token
from database import get_db
import pymongo
from bson import ObjectId
from jose import JWTError, jwt
from config.settings import settings

router = APIRouter(prefix="/api/v1/appointments", tags=["appointments"])

def get_current_user_email(token: str = Depends(oauth2_scheme)):
    credentials_exception = HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Could not validate credentials",
        headers={"WWW-Authenticate": "Bearer"},
    )
    try:
        payload = jwt.decode(token, settings.SECRET_KEY, algorithms=[settings.ALGORITHM])
        email: str = payload.get("sub")
        if email is None:
            raise credentials_exception
        return email
    except JWTError:
        raise credentials_exception

@router.post("/", response_model=dict)
async def create_appointment(
    appointment: AppointmentCreate,
    token: str = Depends(oauth2_scheme)
):
    db = get_db()
    user_email = get_current_user_email(token)
    user = get_user(user_email)

    if not user:
        raise HTTPException(status_code=404, detail="User not found")

    doctor = db.doctors.find_one({"_id": ObjectId(appointment.doctor_id)})
    if not doctor:
        raise HTTPException(status_code=404, detail="Doctor not found")

    existing_appointment = db.appointments.find_one({
        "doctor_id": ObjectId(appointment.doctor_id),
        "date": appointment.date,
        "time_slot": appointment.time_slot,
        "status": "booked"
    })

    if existing_appointment:
        raise HTTPException(
            status_code=409,
            detail="This time slot is already booked"
        )

    appointment_data = {
        "user_id": ObjectId(user.id),
        "doctor_id": ObjectId(appointment.doctor_id),
        "date": appointment.date,
        "time_slot": appointment.time_slot,
        "status": "booked",
        "created_at": datetime.utcnow(),
        "updated_at": datetime.utcnow()
    }

    result = db.appointments.insert_one(appointment_data)

    return {
        "success": True,
        "message": "Appointment booked successfully.",
        "appointment_id": str(result.inserted_id)
    }

# بقیه توابع (GET, PUT, DELETE) مانند کد قبلی

Overwriting doctor_appointment_api/app/appointments.py


main.py

In [ ]:
%%writefile doctor_appointment_api/app/main.py
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from database import connect_to_mongo, close_mongo_connection
from auth import router as auth_router
from appointments import router as appointments_router
import uvicorn

app = FastAPI(title="Doctor Appointment System API")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.include_router(auth_router)
app.include_router(appointments_router)

@app.on_event("startup")
async def startup_db_client():
    connect_to_mongo()

@app.on_event("shutdown")
async def shutdown_db_client():
    close_mongo_connection()

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

Overwriting doctor_appointment_api/app/main.py


API test

In [ ]:
!pip install pyngrok uvicorn fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok
import threading
import uvicorn

# **Step 3: Configure ngrok with your authtoken**
ngrok.set_auth_token("YOUR_AUTHTOKEN") # Replace YOUR_AUTHTOKEN with your actual authtoken

def run_server():
    uvicorn.run("doctor_appointment_api.app.main:app", host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_server)
thread.start()
from pyngrok import ngrok
import threading
import uvicorn

# **Step 3: Configure ngrok with your authtoken**
ngrok.set_auth_token("YOUR_AUTHTOKEN") # Replace YOUR_AUTHTOKEN with your actual authtoken

def run_server():
    uvicorn.run("doctor_appointment_api.app.main:app", host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_server)
thread.start()

from pyngrok import ngrok
import threading
import uvicorn

# **Step 3: Configure ngrok with your authtoken**
ngrok.set_auth_token("YOUR_AUTHTOKEN") # Replace YOUR_AUTHTOKEN with your actual authtoken

def run_server():
    uvicorn.run("doctor_appointment_api.app.main:app", host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_server)
thread.start()

public_url = ngrok.connect(8000)
print("Public URL:2wv70ChDBKfEpsFqHpJEWg8GNKl_7hWD22uBWCzyRd6Za1tfw", public_url)
#agent:  #This line was causing a syntax error.  It appears to be an attempt to define a YAML configuration.  It's commented out as it is not valid Python.
#  authtoken: <your-authtoken>

Exception in thread Thread-31 (run_server):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-20-a40977820086>", line 9, in run_server
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    ret

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

In [ ]:
!ngrok http http://localhost:8080

ERROR:  authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.
ERROR:  Your authtoken: YOUR_AUTHTOKEN
ERROR:  Instructions to install your authtoken are on your ngrok dashboard:
ERROR:  https://dashboard.ngrok.com/get-started/your-authtoken
ERROR:  
ERROR:  ERR_NGROK_105
ERROR:  https://ngrok.com/docs/errors/err_ngrok_105
ERROR:  


In [ ]:
import uvicorn
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Server is running"}

uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
!zip -r DoctorAppointmentAPI.ipynb.zip doctor_appointment_api/

	zip warning: name not matched: doctor_appointment_api/

zip error: Nothing to do! (try: zip -r DoctorAppointmentAPI.ipynb.zip . -i doctor_appointment_api/)


In [ ]:
!zip -r doctor_appointment_api/DoctorAppointmentAPI.ipynb.zip doctor_appointment_api/

	zip warning: name not matched: doctor_appointment_api/

zip error: Nothing to do! (try: zip -r doctor_appointment_api/DoctorAppointmentAPI.ipynb.zip . -i doctor_appointment_api/)


In [ ]:
!pip install pyngrok uvicorn fastapi

In [ ]:
from pyngrok import ngrok
import threading
import uvicorn

def run_server():
    uvicorn.run("doctor_appointment_api.app.main:app", host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_server)
thread.start()

public_url = ngrok.connect(8000)
print("Public URL:", public_url)

Exception in thread Thread-8 (run_server):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-7e2d57c34111>", line 6, in run_server
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    retur

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

In [ ]:
!pip install --force-reinstall pyngrok uvicorn fastapi

  Using cached pyngrok-7.2.8-py3-none-any.whl.metadata (10 kB)
  Using cached uvicorn-0.34.2-py3-none-any.whl.metadata (6.5 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached starlette-0.46.2-py3-none-any.whl.metadata (6.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 5.7 MB/s eta 0:00:00
Using cached pyngrok-7.2.8-py3-none-any.whl (25 kB)
Using cached uvicorn-0.34.2-py3-none-any.whl (62 kB)
Using cached fastapi-0.115.12-py3-none-any.whl (95 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 48.7 MB/s eta 0:00:00
Using cached starlette-0.46.2-py3-none-any.whl (72 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9

In [ ]:
!pip install --force-reinstall fastapi uvicorn pyngrok python-jose[cryptography] passlib[bcrypt] pymongo

  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.2-py3-none-any.whl.metadata (6.5 kB)
  Using cached pyngrok-7.2.8-py3-none-any.whl.metadata (10 kB)
  Using cached starlette-0.46.2-py3-none-any.whl.metadata (6.2 kB)
  Using cached pydantic-2.11.4-py3-none-any.whl.metadata (66 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata 